# Callbacks in Keras

Keras callbacks can help you fix bugs more quickly, and can help you build better models. They can help you visualize how your model’s training is going, and can even help prevent overfitting by implementing early stopping or customizing the learning rate on each iteration.

Keras callbacks return information from a training algorithm while training is taking place. 

> A callback is a set of functions to be applied at given stages of the training procedure. You can use callbacks to get a view on internal states and statistics of the model during training.

## Default Callbacks

### BaseLogger and History

You get these without doing anything special — they’re applied to the model by default. By convention, most of the time you’ll just call model.fit()on a model object without assigning the result to anything. Instead of calling model.fit(), you can write my_history = model.fit(). The my_history variable is assigned a keras.callbacks.History object. The historyproperty of this object is a dict with average accuracy and average loss information for each epoch. You can also inspect the paramsproperty, which is a dict of the parameters used to fit the model.

## ModelCheckpoint

This callback will save your model as a checkpoint file (in hdf5 format) to disk after each successful epoch. You can actually set the output file to be dynamically named based on the epoch. You can also write either the loss value or accuracy value as part of the log’s file name. This is especially handy for models where epochs take an extremely long time, or if you’re running them on AWS spot instances and the spot instance price rises above your maximum bid.

## CSVLogger

t writes a CSV log file containing information about epochs, accuracy, and loss to disk so you can inspect it later. It’s great if you want to roll your own graphs or keep a record of your model training process over time.

## EarlyStopping

One technique to reduce overfitting in neural networks is to use early stopping. Early stopping prevents overtraining of your model by terminating the training process if it’s not really learning anything. This is pretty flexible — you can control what metric to monitor, how much it needs to change to be considered “still learning”, and how many epochs in a row it can falter before the model stops training.

## RemoteMonitor

This callback sends JSON status messages via an HTTP POST method. This could be easily integrated with a pub/sub messaging service like Kafka or a queue like Amazon SQS. For instance, you could setup a lightweight HTTP endpoint and route any POSTed JSON as a payload to an SQS queue. You could then have another process monitor the queue to kick off other processes or to handle specific events.

## LearningRateScheduler

One method is to start with a relatively large value, and decrease it in later training epochs. All you need to do is write a simple function that returns the desired learning rate based on the current epoch, and pass it as the schedule parameter to this callback.

## Tensorboard

By using a TensorBoard callback, logs will be written to a directory that you can then examine with TensorFlow’s excellent TensorBoard visualization tool. It even works (to an extent) if you’re using a backend other than TensorFlow, like Theano, or CNTK.

## LambdaCallback

If you just want to create a basic one, you can use LambdaCallback. This allows you to trigger events when an epoch, batch, or training process begins or ends.

If you want to create something a little more complex, you can create your own callback by inheriting from the keras.callbacks.Callback class. This is only for those who truly want low-level control over how a callback is executed.

# Example of a Callback

One approach to solving this problem is to use early stopping. This involves monitoring the loss on the training dataset and a validation dataset (a subset of the training set not used to fit the model). As soon as loss for the validation set starts to show signs of overfitting, the training process can be stopped.

Early stopping can be used with your model by first ensuring that you have a validation dataset. You can define the validation dataset manually via the validation_data argument to the fit() function, or you can use the validation_split and specify the amount of the training dataset to hold back for validation.

In [1]:
from sklearn.datasets import make_classification
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
X, y = make_classification(n_samples=1000, n_features=4, n_classes=2, random_state=1)
n_features = X.shape[1]

In [5]:
model = Sequential()
model.add(Dense(10, activation='relu', kernel_initializer='he_normal', input_shape=(n_features,)))
model.add(Dense(1, activation='sigmoid'))

In [6]:
model.compile(optimizer='adam', loss='binary_crossentropy')

In [7]:
es = EarlyStopping(monitor='val_loss', patience=5)

In [8]:
history = model.fit(X, y, epochs=200, batch_size=32, verbose=0, validation_split=0.3, callbacks=[es])

In [11]:
print('The model started overfitting the data at '+ str(len(history.history['loss']))+ 'th epoch')

The model started overfitting the data at 70th epoch
